### 1. Chemical space using PCA and molecular fingerprints


---


In [ ]:
from IPython.utils import io
import tqdm.notebook
import os, sys, random
total = 100
with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:
      # Instalar rdkit
      !pip -q install rdkit #.pypi==2021.9.4
      pbar.update(20)
      # Instalar Pillow
      !pip -q install Pillow
      pbar.update(40)
      # Instalar molplotly
      !pip install molplotly
      pbar.update(60)
      # Instalar jupyter-dash
      !pip install jupyter-dash
      pbar.update(80)
      # Instalar el diseño de aplicación dash
      !pip install dash-bootstrap-components
      pbar.update(100)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np
from rdkit.Chem import MACCSkeys
from scipy.spatial.distance import pdist, squareform

In [ ]:
# fragments
url_data = "https://drive.google.com/file/d/12lEtG19vJ0fXS6vvckV9jA3rpbiIfZRi/view?usp=drive_link"
url_data = 'https://drive.google.com/uc?id=' + url_data.split('/')[-2]
data = pd.read_csv(url_data)
data

,ID,All_fragments,All_fragments_h,Counts
0,NPDBEJECOL_All_frag_1,*C(C)C,CCC,34
1,NPDBEJECOL_All_frag_2,*O,O,20
2,NPDBEJECOL_All_frag_3,*C(C)CCC=C(C)C,CCCCC=C(C)C,13
3,NPDBEJECOL_All_frag_4,*CCCC(*)C,CCCCC,13
4,NPDBEJECOL_All_frag_5,*OCC,CCO,11
...,...,...,...,...
195,NPDBEJECOL_All_frag_196,*C(=O)CCCCCCCCCCCCCCC,CCCCCCCCCCCCCCCC=O,1
196,NPDBEJECOL_All_frag_197,*CC/C=C(\C)CCC*,CC/C=C(\C)CCC,1
197,NPDBEJECOL_All_frag_198,*CC/C=C(\C)CCC=C(C)C,CC/C=C(\C)CCC=C(C)C,1
198,NPDBEJECOL_All_frag_199,*CCC/C(C)=C/C/C=C(\C)C=C,C=C/C(C)=C/C/C=C(\C)CCC,1


## Morgan2 for compounds

---


In [ ]:
fps = pd.DataFrame([[int(y) for y in AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(x), 3, nBits=1024).ToBitString()] for x in data["All_fragments_h"]])
fps

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
198,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
# Molecular similarity
SimMat_fps = 1 - pdist(fps[[x for x in range(1024)]], metric="jaccard")#Matriz de similitud_FDA
SimMat_fps
SimMat_fps = squareform(SimMat_fps)
SimMat_fps = 1-SimMat_fps

# Print matrix similarity
SimMat_fps = pd.DataFrame(SimMat_fps)
SimMat_fps

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,1.000000,1.0,0.842105,0.625000,0.571429,0.888889,0.833333,0.888889,0.769231,1.000000,...,0.961538,0.785714,0.896552,0.857143,0.962963,0.869565,0.850000,0.880000,0.900000,0.984127
1,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,0.842105,1.0,1.000000,0.684211,0.800000,0.909091,0.894737,0.909091,0.750000,0.970588,...,0.948718,0.807692,0.756757,0.733333,0.923077,0.750000,0.724138,0.645161,0.794872,0.960000
3,0.625000,1.0,0.684211,1.000000,0.700000,0.916667,0.888889,0.916667,0.733333,1.000000,...,0.965517,0.750000,0.870968,0.826087,0.966667,0.739130,0.818182,0.892857,0.875000,0.984848
4,0.571429,1.0,0.800000,0.700000,1.000000,0.909091,0.714286,0.909091,0.800000,1.000000,...,0.964286,0.733333,0.827586,0.761905,0.965517,0.880000,0.809524,0.846154,0.870968,0.952381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.869565,1.0,0.750000,0.739130,0.880000,0.833333,0.958333,0.833333,0.583333,0.916667,...,0.902439,0.833333,0.837209,0.833333,0.820513,1.000000,0.861111,0.878049,0.891304,0.935065
196,0.850000,1.0,0.724138,0.818182,0.809524,0.913043,0.900000,0.913043,0.807692,0.971429,...,0.950000,0.720000,0.575758,0.440000,0.925000,0.861111,1.000000,0.517241,0.588235,0.960526
197,0.880000,1.0,0.645161,0.892857,0.846154,0.928571,0.920000,0.928571,0.838710,0.975000,...,0.955556,0.843750,0.631579,0.705882,0.909091,0.878049,0.517241,1.000000,0.767442,0.962963
198,0.900000,1.0,0.794872,0.875000,0.870968,0.939394,0.933333,0.939394,0.861111,0.954545,...,0.916667,0.800000,0.674419,0.600000,0.940000,0.891304,0.588235,0.767442,1.000000,0.952941


In [ ]:
# Data Normalized
from sklearn.preprocessing import StandardScaler
data_std = SimMat_fps.copy()
data_std = StandardScaler().fit_transform(data_std)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca_results = pca.fit_transform(data_std)
pca_results

array([[-5.74114669e+00,  5.81120137e+00],
       [ 7.65611742e+00, -2.57565602e+00],
       [-7.57540160e+00,  4.44114476e+00],
       [-8.48176909e+00,  4.76233819e+00],
       [-6.76648527e+00,  9.37645244e+00],
       [-2.29827113e+00, -4.12584742e+00],
       [ 1.96567954e+00,  4.11855876e+00],
       [-2.29827113e+00, -4.12584742e+00],
       [-1.11151867e+01, -2.83597431e+00],
       [ 7.83892903e+00, -9.69082618e+00],
       [-8.07768402e+00,  2.37383211e+00],
       [-6.37421709e+00, -3.07108915e-01],
       [-1.19813546e+01, -9.28147922e+00],
       [-5.65858292e+00,  2.37072491e+00],
       [ 1.02480597e+01, -3.36297159e+00],
       [-8.10512106e+00,  2.22273305e+00],
       [-2.41383216e+00,  3.46701707e+00],
       [-8.14822742e+00,  6.89774426e+00],
       [ 6.78886004e+00,  2.70745940e+00],
       [-8.36627795e+00,  3.92124034e+00],
       [ 8.28770012e+00,  2.94227079e+00],
       [-7.51256547e+00,  3.56582395e+00],
       [ 9.66712244e+00,  4.02213424e+00],
       [ 9.

In [ ]:
# Complementary information
label = data[["ID", "All_fragments_h"]]
label = label.to_numpy()

In [ ]:
# Concat arrays de numpy
arr = np.concatenate((label, pca_results), axis = 1)
# Crreate a new dataframe
pca_dataset = pd.DataFrame(data=arr, columns = ["ID", "All_fragments_h",'component1', 'component2'])
pca_dataset.head(2)

,ID,All_fragments_h,component1,component2
0,NPDBEJECOL_All_frag_1,CCC,-5.741147,5.811201
1,NPDBEJECOL_All_frag_2,O,7.656117,-2.575656


In [ ]:
# Determine explained variance using explained_variance_ration_ attribute
exp_var_pca = pca.explained_variance_ratio_
exp_var_pca

array([0.27511462, 0.12310317])

In [ ]:
#Graph
import plotly.express as px
import molplotly
fig_pca = px.scatter(pca_dataset,
                            x='component1',
                            y='component2',
                            title='PCA',
                            labels={'component1': f'PC1 {round(exp_var_pca[0] * 100, 2)}%',
                                    'component2': f'PC2 {round(exp_var_pca[1] * 100, 2)}%'},
                            width=700,
                            height=500)

fig_pca.update_traces(marker=dict(color='purple'))

app_marker = molplotly.add_molecules(fig=fig_pca,
                                         df=pca_dataset,
                                         smiles_col='All_fragments_h',
                                         title_col='ID')
#fig_pca.show()
#app_marker.run_server(mode='inline', port=8060, height=1000)
app_marker.run(port=8060)

/usr/local/lib/python3.11/dist-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

## MACCS keys for compounds

---



In [ ]:
# Recalculate the MACCS keys
fps = pd.DataFrame([[int(y) for y in MACCSkeys.GenMACCSKeys(Chem.MolFromSmiles(x)).ToBitString()] for x in data['All_fragments_h']])
# Molecular similarity
SimMat_fps = 1 - pdist(fps[[x for x in range(167)]], metric="jaccard")#Matriz de similitud_FDA
SimMat_fps
SimMat_fps = squareform(SimMat_fps)
SimMat_fps = 1-SimMat_fps

# Print matrix similarity
SimMat_fps = pd.DataFrame(SimMat_fps)
SimMat_fps

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,1.000000,1.000000,0.642857,0.555556,0.727273,0.857143,0.888889,0.857143,0.727273,1.000000,...,0.944444,0.823529,0.826087,0.809524,0.923077,0.800000,0.750000,0.687500,0.750000,0.976190
1,1.000000,1.000000,1.000000,1.000000,0.777778,0.750000,0.600000,0.750000,0.900000,0.833333,...,0.933333,0.866667,0.909091,0.900000,0.900000,0.928571,1.000000,1.000000,1.000000,0.947368
2,0.642857,1.000000,1.000000,0.428571,0.850000,0.937500,0.944444,0.937500,0.647059,1.000000,...,0.962963,0.739130,0.434783,0.521739,0.850000,0.578947,0.187500,0.125000,0.294118,0.980392
3,0.555556,1.000000,0.428571,1.000000,0.785714,0.900000,0.916667,0.900000,0.454545,1.000000,...,0.952381,0.647059,0.636364,0.600000,0.785714,0.500000,0.466667,0.500000,0.466667,0.977778
4,0.727273,0.777778,0.850000,0.785714,1.000000,0.800000,0.600000,0.800000,0.714286,0.923077,...,0.850000,0.666667,0.590909,0.550000,0.875000,0.777778,0.857143,0.863636,0.857143,0.825000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.800000,0.928571,0.578947,0.500000,0.777778,0.769231,0.875000,0.769231,0.307692,0.875000,...,0.875000,0.526316,0.541667,0.565217,0.625000,1.000000,0.444444,0.473684,0.526316,0.914894
196,0.750000,1.000000,0.187500,0.466667,0.857143,0.941176,0.947368,0.941176,0.666667,1.000000,...,0.964286,0.750000,0.318182,0.409091,0.857143,0.444444,1.000000,0.062500,0.125000,0.980769
197,0.687500,1.000000,0.125000,0.500000,0.863636,0.944444,0.950000,0.944444,0.684211,1.000000,...,0.965517,0.760000,0.347826,0.434783,0.863636,0.473684,0.062500,1.000000,0.176471,0.981132
198,0.750000,1.000000,0.294118,0.466667,0.857143,0.941176,0.947368,0.941176,0.666667,1.000000,...,0.964286,0.750000,0.391304,0.409091,0.857143,0.526316,0.125000,0.176471,1.000000,0.980769


In [ ]:
# Data Normalized
from sklearn.preprocessing import StandardScaler
data_std = SimMat_fps.copy()
data_std = StandardScaler().fit_transform(data_std)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca_results = pca.fit_transform(data_std)
pca_results

array([[-1.94331678e+00,  9.70509532e+00],
       [ 9.21035784e+00,  1.21056187e+01],
       [-1.11133291e+01, -3.74301330e-01],
       [-9.58229868e+00,  3.48390938e+00],
       [ 2.17233654e+00,  5.46955881e+00],
       [ 4.69653530e+00,  1.35303326e+01],
       [ 6.96607410e+00,  9.25084011e+00],
       [ 4.69653530e+00,  1.35303326e+01],
       [-7.42711487e+00,  3.96594699e+00],
       [ 1.04779124e+01,  1.00589224e+01],
       [-5.93631031e+00,  6.27081469e+00],
       [-4.99998481e+00, -4.81446080e+00],
       [-1.17194230e+01, -1.24087342e+00],
       [-4.67748988e+00, -2.61205324e+00],
       [ 1.64732443e+01, -2.43546898e+00],
       [-1.23463358e+01, -2.68360757e-01],
       [ 1.72782629e+00,  5.01741273e+00],
       [-8.19209581e+00,  5.03375642e+00],
       [ 1.28384653e+01, -5.51091273e+00],
       [-1.16101584e+01,  1.08619499e+00],
       [ 1.09020848e+01, -6.93325852e+00],
       [-1.80305032e+00,  1.00749921e+01],
       [ 1.32669060e+01, -5.10304449e+00],
       [ 1.

In [ ]:
# Complementary information
label = data[["ID", "All_fragments_h"]]
label = label.to_numpy()

In [ ]:
# Concat arrays de numpy
arr = np.concatenate((label, pca_results), axis = 1)
# Crreate a new dataframe
pca_dataset = pd.DataFrame(data=arr, columns = ["ID", "All_fragments_h",'component1', 'component2'])
pca_dataset.head(2)

,ID,All_fragments_h,component1,component2
0,NPDBEJECOL_All_frag_1,CCC,-1.943317,9.705095
1,NPDBEJECOL_All_frag_2,O,9.210358,12.105619


In [ ]:
# Determine explained variance using explained_variance_ration_ attribute
exp_var_pca = pca.explained_variance_ratio_
exp_var_pca

array([0.44565445, 0.14473958])

In [ ]:
#Graph
import plotly.express as px
import molplotly
fig_pca = px.scatter(pca_dataset,
                            x='component1',
                            y='component2',
                            title='PCA',
                            labels={'component1': f'PC1 {round(exp_var_pca[0] * 100, 2)}%',
                                    'component2': f'PC2 {round(exp_var_pca[1] * 100, 2)}%'},
                            width=700,
                            height=500)

fig_pca.update_traces(marker=dict(color='red'))

app_marker = molplotly.add_molecules(fig=fig_pca,
                                         df=pca_dataset,
                                         smiles_col='All_fragments_h',
                                         title_col='ID')
#fig_pca.show()
#app_marker.run_server(mode='inline', port=8060, height=1000)
app_marker.run(port=8060)

/usr/local/lib/python3.11/dist-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>